In [1]:
import numpy as np
import pandas as pd
import itertools
import random
from tqdm import tqdm
import re
from wordle_helpers import anagram_scoring, all_anagram_scoring

import os
# where to save the data
ROOT_DIR = "."
FOLDER_NAME = "data"
PATH_TO_FOLDER = os.path.join(ROOT_DIR, FOLDER_NAME)
os.makedirs(PATH_TO_FOLDER, exist_ok=True)

## Load data

In [2]:
anagrams = pd.read_csv("./data/anagrams.csv")

In [3]:
anagrams

,word,word_length,anagrams
0,abode,5,"abode, adobe"
1,abort,5,"abort, tabor"
2,acred,5,"acred, cader"
3,acrid,5,"acrid, caird"
4,adept,5,"adept, pated"
...,...,...,...
382,worth,5,"worth, wroth, whort"
383,wrath,5,"wrath, thraw"
384,wreak,5,"wreak, waker"
385,wrote,5,"wrote, tower"


In [4]:
anagrams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   word         387 non-null    object
 1   word_length  387 non-null    int64 
 2   anagrams     387 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.2+ KB


In [5]:
anagrams["word"].nunique()

387

## Number of anagrams

In [6]:
my_list_of_anagrams = [pair.split(", ") for pair in anagrams["anagrams"]]

In [7]:
my_list_of_anagrams[:5]

[['abode', 'adobe'],
 ['abort', 'tabor'],
 ['acred', 'cader'],
 ['acrid', 'caird'],
 ['adept', 'pated']]

In [8]:
# num_anagrams_per_list = [len(lst) for lst in my_list_of_anagrams]

In [9]:
vec = np.vectorize(len)
num_anagrams_per_list = vec(np.char.split(np.array(anagrams["anagrams"], dtype=str), sep=", "))
num_anagrams_per_list[:5]

array([2, 2, 2, 2, 2])

In [10]:
anagrams["num_anagrams"] = num_anagrams_per_list

In [11]:
anagrams.head()

,word,word_length,anagrams,num_anagrams
0,abode,5,"abode, adobe",2
1,abort,5,"abort, tabor",2
2,acred,5,"acred, cader",2
3,acrid,5,"acrid, caird",2
4,adept,5,"adept, pated",2


In [12]:
anagrams_copy = anagrams.copy()
anagrams_copy.head()

,word,word_length,anagrams,num_anagrams
0,abode,5,"abode, adobe",2
1,abort,5,"abort, tabor",2
2,acred,5,"acred, cader",2
3,acrid,5,"acrid, caird",2
4,adept,5,"adept, pated",2


In [13]:
# using assign to create new column
# (anagrams.
#  assign(num_anagrams = [len(lst) for lst in [pair.split(", ") for pair in anagrams["anagrams"]]]))

# (anagrams.
#  assign(num_anagrams = vec(np.char.split(np.array(anagrams["anagrams"], dtype=str),sep=", "))))

In [14]:
anagrams.loc[anagrams["num_anagrams"] == anagrams["num_anagrams"].max()]

,word,word_length,anagrams,num_anagrams
292,steal,5,"steal, tales, slate, stale, satle, stela, astel",7


In [15]:
anagrams.loc[anagrams["num_anagrams"] == anagrams["num_anagrams"].max()].index[0]

292

In [16]:
indx = np.argmax(num_anagrams_per_list)
indx

292

In [17]:
np.argmax(anagrams["num_anagrams"])

292

In [18]:
words = my_list_of_anagrams[indx]
words

['steal', 'tales', 'slate', 'stale', 'satle', 'stela', 'astel']

## Scoring anagrams

In [19]:
challenge = words[0]
challenge

'steal'

In [20]:
guess_anagrams = words[1:]
guess_anagrams

['tales', 'slate', 'stale', 'satle', 'stela', 'astel']

In [21]:
data = []
for guess in guess_anagrams:
    paired = zip(challenge, guess)
    scores = np.zeros(5, dtype=np.int8)
    for x, (i, j) in enumerate(paired):
#         print((x, (i, j)))
        if i == j:
            scores[x] = 1
        elif i != j and j in challenge:
            scores[x] = 0
        else:
            scores[x] = -1
            
    temp = {
        "challenge_word": challenge,
        "guess": guess,
        "positional_scores": re.sub(r"[\[\]]", "", str(scores)).replace("\n", ","),
        "overall_scores": np.sum(scores, dtype=np.int8),
    }
    data.append(temp)

In [22]:
df = pd.DataFrame(data)

In [23]:
df

,challenge_word,guess,positional_scores,overall_scores
0,steal,tales,0 0 0 0 0,0
1,steal,slate,1 0 0 0 0,1
2,steal,stale,1 1 0 0 0,2
3,steal,satle,1 0 0 0 0,1
4,steal,stela,1 1 1 0 0,3
5,steal,astel,0 0 0 0 1,1


In [24]:
# 3 anagram words
anagrams.loc[anagrams["num_anagrams"] == 3].iloc[:5]

,word,word_length,anagrams,num_anagrams
11,alien,5,"alien, aline, anile",3
22,ample,5,"ample, maple, pelma",3
30,argol,5,"argol, algor, orgal",3
31,argon,5,"argon, orang, angor",3
32,aries,5,"aries, serai, aesir",3


### Scoring each anagram against itself

In [25]:
data2 = []
num_guesses = len(words)
counter = 0

while counter < num_guesses:
    for word in words:
        guess = words[counter]
        combo_guess_word = zip(guess, word)
        scores = np.zeros(5, dtype=np.int8)
        for x, (i, j) in enumerate(combo_guess_word):
            if i == j:
                scores[x] = 1
            elif i != j and j in word:
                scores[x] = 0
            else:
                scores[x] = -1
        data2.append(re.sub(r"[\[\]]", "", str(scores)).replace("\n", ","))
    counter += 1   

In [26]:
data2[:7]

['1 1 1 1 1',
 '0 0 0 0 0',
 '1 0 0 0 0',
 '1 1 0 0 0',
 '1 0 0 0 0',
 '1 1 1 0 0',
 '0 0 0 0 1']

In [27]:
len(data2), num_guesses

(49, 7)

In [28]:
# Every 7th value (num guesses) in data2 represents the next guess word scored against the challenge word
# so we can reshape data2 to be a 7x7 array
np.array(data2).reshape(num_guesses, num_guesses)

array([['1 1 1 1 1', '0 0 0 0 0', '1 0 0 0 0', '1 1 0 0 0', '1 0 0 0 0',
        '1 1 1 0 0', '0 0 0 0 1'],
       ['0 0 0 0 0', '1 1 1 1 1', '0 0 0 0 0', '0 0 0 0 0', '0 1 0 0 0',
        '0 0 0 0 0', '0 0 0 1 0'],
       ['1 0 0 0 0', '0 0 0 0 0', '1 1 1 1 1', '1 0 1 0 1', '1 0 0 0 1',
        '1 0 0 0 0', '0 0 0 0 0'],
       ['1 1 0 0 0', '0 0 0 0 0', '1 0 1 0 1', '1 1 1 1 1', '1 0 0 1 1',
        '1 1 0 1 0', '0 0 0 0 0'],
       ['1 0 0 0 0', '0 1 0 0 0', '1 0 0 0 1', '1 0 0 1 1', '1 1 1 1 1',
        '1 0 0 1 0', '0 0 1 0 0'],
       ['1 1 1 0 0', '0 0 0 0 0', '1 0 0 0 0', '1 1 0 1 0', '1 0 0 1 0',
        '1 1 1 1 1', '0 0 0 0 0'],
       ['0 0 0 0 1', '0 0 0 1 0', '0 0 0 0 0', '0 0 0 0 0', '0 0 1 0 0',
        '0 0 0 0 0', '1 1 1 1 1']], dtype='<U9')

In [29]:
df2 = pd.DataFrame(np.array(data2).reshape(7, 7), columns=words, index=words)

In [30]:
df2

,steal,tales,slate,stale,satle,stela,astel
steal,1 1 1 1 1,0 0 0 0 0,1 0 0 0 0,1 1 0 0 0,1 0 0 0 0,1 1 1 0 0,0 0 0 0 1
tales,0 0 0 0 0,1 1 1 1 1,0 0 0 0 0,0 0 0 0 0,0 1 0 0 0,0 0 0 0 0,0 0 0 1 0
slate,1 0 0 0 0,0 0 0 0 0,1 1 1 1 1,1 0 1 0 1,1 0 0 0 1,1 0 0 0 0,0 0 0 0 0
stale,1 1 0 0 0,0 0 0 0 0,1 0 1 0 1,1 1 1 1 1,1 0 0 1 1,1 1 0 1 0,0 0 0 0 0
satle,1 0 0 0 0,0 1 0 0 0,1 0 0 0 1,1 0 0 1 1,1 1 1 1 1,1 0 0 1 0,0 0 1 0 0
stela,1 1 1 0 0,0 0 0 0 0,1 0 0 0 0,1 1 0 1 0,1 0 0 1 0,1 1 1 1 1,0 0 0 0 0
astel,0 0 0 0 1,0 0 0 1 0,0 0 0 0 0,0 0 0 0 0,0 0 1 0 0,0 0 0 0 0,1 1 1 1 1


### Take the anagram dataframe and randomly select a word to display the scoring

In [31]:
# show score for random anagrams
anagram_scoring_df = anagram_scoring(anagrams)
anagram_scoring_df

,poker,proke
poker,1 1 1 1 1,1 0 0 0 0
proke,1 0 0 0 0,1 1 1 1 1


### Score all anagrams against each other. The scoring partners which aren't anagrams will contain `-1s`  

In [ ]:
all_anagram_scoring_df = all_anagram_scoring(anagrams)

 95%|████████████████████████████████████████████████████████████████████████████▊    | 637/672 [00:20<00:01, 31.53it/s]

In [ ]:
all_anagram_scoring_df.info(memory_usage="deep")

In [ ]:
all_anagram_scoring_df

In [ ]:
all_anagram_scoring_df.to_csv("./data/all_anagram_scoring.csv", index=False)

## 5 bit binary  

Scoring of anagrams produces 5 bit binary with range from `00000` to `11111` `(0 to 31)`.  
- `00000`, all letter are in wrong place
- `11111`, letters matched correctly

In [ ]:
df2

In [ ]:
int("11111", 2)

In [ ]:
int("00000", 2)

In [ ]:
int("10000", 2)

In [ ]:
int("10101", 2)

**`stale` contains 24 bits of information necessary to describe `steal`**

In [ ]:
int("11000", 2)

In [ ]:
int("00100", 2)

In [ ]:
df2.loc[df2.index.isin(["stale"])]

In [ ]:
stale = df2.loc[df2.index.isin(["stale"])].T
stale

In [ ]:
df2.loc[df2.index.isin(["stale"])].squeeze()

In [ ]:
pd.DataFrame(df2.loc[df2.index.isin(["stale"])].squeeze())

In [ ]:
stale_binaries = df2.loc[df2.index.isin(["stale"])].squeeze().tolist()
stale_binaries

In [ ]:
stale_binaries[0]

In [ ]:
"".join(stale_binaries[0].split())

In [ ]:
int("".join(stale_binaries[0].split()), 2)

In [ ]:
stale_binaries = ["".join(x.split()) for x in stale_binaries]
stale_binaries

In [ ]:
[int(binary, 2) for binary in stale_binaries]

In [ ]:
scare = all_anagram_scoring_df.loc[all_anagram_scoring_df.index.isin(["scare"])].squeeze()
scare

In [ ]:
#  you can find the scores for scare by filtering out those with -1
cols = [(idx, binary) for idx, binary in enumerate(scare) if "-1" not in binary]
cols

In [ ]:
scare_cols = [col[0] for col in cols]
scare_cols

In [ ]:
all_anagram_scoring_df.iloc[scare_cols, scare_cols]

In [ ]:
# remove spaces
all_anagram_scoring_df.iloc[scare_cols, scare_cols].applymap(lambda x: "".join(x.split()))

In [ ]:
# convert to int
(all_anagram_scoring_df.iloc[scare_cols, scare_cols]
 .applymap(lambda x: "".join(x.split()))
 .applymap(lambda x: int(x, 2)))

In [ ]:
(all_anagram_scoring_df.iloc[scare_cols, scare_cols]
 .applymap(lambda x: "".join(x.split()))
 .applymap(lambda x: int(x, 2))
 .assign(sum_across=lambda x: x.sum())
 .astype(np.int8))

## Anagram cipher lookup table  
With `itertools.zip_longest` you can create an encode/decode lookup table for 4, 5 and 6 letter anagrams.

In [ ]:
list(itertools.zip_longest("taper", "slip", fillvalue="*"))

In [ ]:
list(itertools.zip_longest("taper", "scare"))